In [9]:
import numpy
import matplotlib.pyplot
import scipy.special
%matplotlib inline 

In [10]:
class neuralNetwork:

    #initialise the neural network
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate):
        
        self.iNodes=inputNodes
        self.hNodes=hiddenNodes
        self.oNodes=outputNodes
        
        #link weight matrices, wih and who
        #weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        #example
            # w11 w21
            # w12 w22
            # ...
        
        self.wih=numpy.random.normal(0.0, pow(self.hNodes, -0.5), (self.hNodes, self.iNodes)) # The -0.5 is to keep the array between -1 and 1
        self.woh=numpy.random.normal(0.0, pow(self.oNodes, -0.5), (self.oNodes, self.hNodes))

        #learning rate
        self.lr=learningRate
        
        #activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
                  
        pass
    
    #train the neural network
    def train(self, inputs_list, targets_list):
        #convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate the signals emerging from the hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        #calculate signals into final output layer
        final_inputs=numpy.dot(self.woh, hidden_outputs)
        #calculate the signals emerging from final output layer
        final_outputs=self.activation_function(final_inputs)
        
        # error is the (target - actual)
        output_errors = targets - final_outputs
        
        #hidden layer error is the output_errors, split by weights, recombined at hidden nodes (backpropagation)
        hidden_errors=numpy.dot(self.woh.T, output_errors)
        
        #update the weights for the links between the hidden and output layers
        self.woh+= self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        #update the weights for the links between the input and hidden layers
        self.wih+= self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    #query the neural network
    def query(self, inputs_list):
        #convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #calculate the signalas emerging from hidden layer
        hidden_outputs=self.activation_function(hidden_inputs)
        #calculate signals into final output layer
        final_inputs=numpy.dot(self.woh, hidden_outputs)
        #calculate the sginals emerging from the final output layer
        final_outputs=self.activation_function(final_inputs)
        
        return final_outputs

In [11]:
# number of input, hidden and output nodes
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

# learning rate is 0.3
learning_rate = 0.3

# instance an object of neural network
n= neuralNetwork( input_nodes, hidden_nodes, output_nodes, learning_rate)


In [12]:
#load the mnist training data CSV file inot a list
training_data_file=open("docs/mnist_train_100.csv", 'r')
training_data_list=training_data_file.readlines()
training_data_file.close()

In [13]:
#train the neural network

#epochs is the number of times the training data set is used for training
epochs=5

for e in range(epochs):
        
    for record in training_data_list:
        #split the record by the ','
        all_values = record.split(',')
        #scale and shift the inputs
        inputs=(numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        #create the target output values (all 0.01 except the desired label which is 0.99)
        targets=numpy.zeros(output_nodes) + 0.01
        #all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
    pass
pass


In [14]:
#load the mnist test data CSV file into a list
test_data_file=open("docs/mnist_test_10.csv", 'r')
test_data_list=test_data_file.readlines()
test_data_file.close()

In [16]:
#test the neural network

#scoreboard for how well the network performs, initially empty
scorecard = []

#go through all the records in the test data set
for record in test_data_list:
    #split the record by the commas
    all_values=record.split(',')
    #correct answer is first value
    correct_label = int(all_values[0])
    print(correct_label, "Correct label")
    #scale and shift the inputs
    inputs=(numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    #query the network
    outputs=n.query(inputs)
    #the index of the highest value corresponds to the label
    label=numpy.argmax(outputs)
    print(label, "network's answer")
    # append correct or incorrect to list
    if (label == correct_label):
        #network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        #network's answer does not match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
pass

7 Correct label
7 network's answer
2 Correct label
2 network's answer
1 Correct label
1 network's answer
0 Correct label
0 network's answer
4 Correct label
4 network's answer
1 Correct label
1 network's answer
4 Correct label
9 network's answer
9 Correct label
4 network's answer
5 Correct label
4 network's answer
9 Correct label
9 network's answer


In [20]:
print(scorecard)
#calculate the performance score, the fraction of correc answers 
scorecard_array = numpy.asarray(scorecard)
performance = (scorecard_array.sum() / scorecard_array.size ) * 100.0
print("Performance = ", performance,"%")

[1, 1, 1, 1, 1, 1, 0, 0, 0, 1]
Performance =  70.0 %
